<a href="https://colab.research.google.com/github/pauloacs/DeepCFD/blob/main/Navier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import contextlib
import functools
import os
import time

import numpy as np
import pandas as pd
import scipy as sp
from six.moves import urllib
from sklearn import preprocessing


import tensorflow_probability as tfp

# CUSTOM_lbfgs

In [ ]:
#%% Adapted from https://github.com/yaroslavvb/stuff/blob/master/eager_lbfgs/eager_lbfgs.py

import tensorflow as tf
import numpy as np
import time

# Time tracking functions
global_time_list = []
global_last_time = 0
def reset_time():
  global global_time_list, global_last_time
  global_time_list = []
  global_last_time = time.perf_counter()
  
def record_time():
  global global_last_time, global_time_list
  new_time = time.perf_counter()
  global_time_list.append(new_time - global_last_time)
  global_last_time = time.perf_counter()
  #print("step: %.2f"%(global_time_list[-1]*1000))

def last_time():
  """Returns last interval records in millis."""
  global global_last_time, global_time_list
  if global_time_list:
    return 1000 * global_time_list[-1]
  else:
    return 0

def dot(a, b):
  """Dot product function since TensorFlow doesn't have one."""
  return tf.reduce_sum(a*b)

def verbose_func(s):
  print(s)

final_loss = None
times = []
def lbfgs(opfunc, x, config, state, do_verbose, log_fn):
  """port of lbfgs.lua, using TensorFlow eager mode.
  """

  if config.maxIter == 0:
    return

  global final_loss, times
  
  maxIter = config.maxIter
  maxEval = config.maxEval or maxIter*1.25
  tolFun = config.tolFun or 1e-5
  tolX = config.tolX or 1e-19
  nCorrection = config.nCorrection or 100
  lineSearch = config.lineSearch
  lineSearchOpts = config.lineSearchOptions
  learningRate = config.learningRate or 1
  isverbose = config.verbose or False

  # verbose function
  if isverbose:
    verbose = verbose_func
  else:
    verbose = lambda x: None

    # evaluate initial f(x) and df/dx
  f, g = opfunc(x)

  f_hist = [f]
  currentFuncEval = 1
  state.funcEval = state.funcEval + 1
  p = g.shape[0]

  # check optimality of initial point
  tmp1 = tf.abs(g)
  if tf.reduce_sum(tmp1) <= tolFun:
    verbose("optimality condition below tolFun")
    return x, f_hist

  # optimize for a max of maxIter iterations
  nIter = 0
  times = []
  while nIter < maxIter:
    start_time = time.time()
    
    # keep track of nb of iterations
    nIter = nIter + 1
    state.nIter = state.nIter + 1

    ############################################################
    ## compute gradient descent direction
    ############################################################
    if state.nIter == 1:
      d = -g
      old_dirs = []
      old_stps = []
      Hdiag = 1
    else:
      # do lbfgs update (update memory)
      y = g - g_old
      s = d*t
      ys = dot(y, s)
      
      if ys > 1e-10:
        # updating memory
        if len(old_dirs) == nCorrection:
          # shift history by one (limited-memory)
          del old_dirs[0]
          del old_stps[0]

        # store new direction/step
        old_dirs.append(s)
        old_stps.append(y)

        # update scale of initial Hessian approximation
        Hdiag = ys/dot(y, y)

      # compute the approximate (L-BFGS) inverse Hessian 
      # multiplied by the gradient
      k = len(old_dirs)

      # need to be accessed element-by-element, so don't re-type tensor:
      ro = [0]*nCorrection
      for i in range(k):
        ro[i] = 1/dot(old_stps[i], old_dirs[i])
        

      # iteration in L-BFGS loop collapsed to use just one buffer
      # need to be accessed element-by-element, so don't re-type tensor:
      al = [0]*nCorrection

      q = -g
      for i in range(k-1, -1, -1):
        al[i] = dot(old_dirs[i], q) * ro[i]
        q = q - al[i]*old_stps[i]

      # multiply by initial Hessian
      r = q*Hdiag
      for i in range(k):
        be_i = dot(old_stps[i], r) * ro[i]
        r += (al[i]-be_i)*old_dirs[i]
        
      d = r
      # final direction is in r/d (same object)

    g_old = g
    f_old = f
    
    ############################################################
    ## compute step length
    ############################################################
    # directional derivative
    gtd = dot(g, d)

    # check that progress can be made along that direction
    if gtd > -tolX:
      verbose("Can not make progress along direction.")
      break

    # reset initial guess for step size
    if state.nIter == 1:
      tmp1 = tf.abs(g)
      t = min(1, 1/tf.reduce_sum(tmp1))
    else:
      t = learningRate


    # optional line search: user function
    lsFuncEval = 0
    if lineSearch and isinstance(lineSearch) == types.FunctionType:
      # perform line search, using user function
      f,g,x,t,lsFuncEval = lineSearch(opfunc,x,t,d,f,g,gtd,lineSearchOpts)
      f_hist.append(f)
    else:
      # no line search, simply move with fixed-step
      x += t*d
      
      if nIter != maxIter:
        # re-evaluate function only if not in last iteration
        # the reason we do this: in a stochastic setting,
        # no use to re-evaluate that function here
        f, g = opfunc(x)
        lsFuncEval = 1
        f_hist.append(f)


    # update func eval
    currentFuncEval = currentFuncEval + lsFuncEval
    state.funcEval = state.funcEval + lsFuncEval

    ############################################################
    ## check conditions
    ############################################################
    if nIter == maxIter:
      break

    if currentFuncEval >= maxEval:
      # max nb of function evals
      verbose('max nb of function evals')
      break

    tmp1 = tf.abs(g)
    if tf.reduce_sum(tmp1) <=tolFun:
      # check optimality
      verbose('optimality condition below tolFun')
      break
    
    tmp1 = tf.abs(d*t)
    if tf.reduce_sum(tmp1) <= tolX:
      # step size below tolX
      verbose('step size below tolX')
      break

    if tf.abs(f-f_old) < tolX:
      # function value changing less than tolX
      verbose('function value changing less than tolX'+str(tf.abs(f-f_old)))
      break

    if do_verbose:
      log_fn(nIter, f.numpy(), True)
      #print("Step %3d loss %6.5f msec %6.3f"%(nIter, f.numpy(), last_time()))
      record_time()
      times.append(last_time())

    if nIter == maxIter - 1:
      final_loss = f.numpy()


  # save state
  state.old_dirs = old_dirs
  state.old_stps = old_stps
  state.Hdiag = Hdiag
  state.g_old = g_old
  state.f_old = f_old
  state.t = t
  state.d = d

  return x, f_hist, currentFuncEval

# dummy/Struct gives Lua-like struct object with 0 defaults
class dummy(object):
  pass

class Struct(dummy):
  def __getattribute__(self, key):
    if key == '__dict__':
      return super(dummy, self).__getattribute__('__dict__')
    return self.__dict__.get(key, 0)


# NN


In [ ]:
import tensorflow as tf
import numpy as np


class NeuralNetwork(object):
    def __init__(self, hp, logger, ub, lb):

        layers = hp["layers"]

        # Setting up the optimizers with the hyper-parameters
        # self.nt_config = Struct()
        # self.nt_config.learningRate = hp["nt_lr"]
        # self.nt_config.maxIter = hp["nt_epochs"]
        # self.nt_config.nCorrection = hp["nt_ncorr"]
        # self.nt_config.tolFun = 1.0 * np.finfo(float).eps
        self.nt_learningRate = hp["nt_lr"]
        self.nt_maxIter = hp["nt_epochs"]
        self.nt_nCorrection = hp["nt_ncorr"]
        self.nt_tolFun = 1.0 * np.finfo(float).eps
        self.tf_epochs = hp["tf_epochs"]
        self.tf_optimizer = tf.keras.optimizers.Adam(
            learning_rate=hp["tf_lr"],
            beta_1=hp["tf_b1"],
            epsilon=hp["tf_eps"])

        self.dtype = "float64"
        # Descriptive Keras model
        tf.keras.backend.set_floatx(self.dtype)
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.InputLayer(input_shape=(layers[0],)))
        self.model.add(tf.keras.layers.Lambda(
            lambda X: 2.0*(X - lb)/(ub - lb) - 1.0))
        for width in layers[1:-1]:
            self.model.add(tf.keras.layers.Dense(
                width, activation=tf.nn.tanh,
                kernel_initializer="glorot_normal"))
        self.model.add(tf.keras.layers.Dense(
                layers[-1], activation=None,
                kernel_initializer="glorot_normal"))

        # Computing the sizes of weights/biases for future decomposition
        self.sizes_w = []
        self.sizes_b = []
        for i, width in enumerate(layers):
            if i != 1:
                self.sizes_w.append(int(width * layers[1]))
                self.sizes_b.append(int(width if i != 0 else layers[1]))

        self.logger = logger

    # Defining custom loss
    # @tf.function
    # def loss(self, X, u, v):
    #     return tf.reduce_mean(tf.square(u - u_pred)) + tf.reduce_mean(tf.square(v - v_pred))

    @tf.function
    def grad(self, X, u, v):
        with tf.GradientTape() as tape:
            loss_value = self.loss(X, u, v)
        grads = tape.gradient(loss_value, self.wrap_training_variables())
        return loss_value, grads

    def wrap_training_variables(self):
        var = self.model.trainable_variables
        return var

    def get_params(self, numpy=False):
        return []

    def get_weights(self, convert_to_tensor=True):
        w = []
        for layer in self.model.layers[1:]:
            weights_biases = layer.get_weights()
            weights = weights_biases[0].flatten()
            biases = weights_biases[1]
            w.extend(weights)
            w.extend(biases)
        if convert_to_tensor:
            w = self.tensor(w)
        return w

    def set_weights(self, w):
        for i, layer in enumerate(self.model.layers[1:]):
            start_weights = sum(self.sizes_w[:i]) + sum(self.sizes_b[:i])
            end_weights = sum(self.sizes_w[:i+1]) + sum(self.sizes_b[:i])
            weights = w[start_weights:end_weights]
            w_div = int(self.sizes_w[i] / self.sizes_b[i])
            weights = tf.reshape(weights, [w_div, self.sizes_b[i]])
            biases = w[end_weights:end_weights + self.sizes_b[i]]
            weights_biases = [weights, biases]
            layer.set_weights(weights_biases)

    # def make_val_and_grad_fn(value_fn):
    #   @functools.wraps(value_fn)
    #   def val_and_grad(w):
    #     self.set_weights(w)
    #     return tfp.math.value_and_gradient(value_fn,self.wrap_training_variables())
    #   return val_and_grad

    def get_loss_and_flat_grad(self, X, u, v):
        def loss_and_flat_grad(w):
            with tf.GradientTape() as tape:
                self.set_weights(w)
                loss_value = self.loss(X, u, v)
            grad = tape.gradient(loss_value, self.wrap_training_variables())
            grad_flat = []
            for g in grad:
                grad_flat.append(tf.reshape(g, [-1]))
            grad_flat = tf.concat(grad_flat, 0)
            return loss_value, grad_flat

        return loss_and_flat_grad

    
    def tf_optimization(self, X_u, u, v):
        self.logger.log_train_opt("Adam")
        for epoch in range(self.tf_epochs):
            loss_value = self.tf_optimization_step(X_u, u, v)
            self.logger.log_train_epoch(epoch, loss_value)

    @tf.function
    def tf_optimization_step(self, X_u, u, v):
        loss_value, grads = self.grad(X_u, u, v)
        self.tf_optimizer.apply_gradients(
                zip(grads, self.wrap_training_variables()))
        return loss_value
    
    #@tf.function
    def nt_optimization(self, X, u, v):
        self.logger.log_train_opt("LBFGS")
        loss_and_flat_grad = self.get_loss_and_flat_grad( X, u, v)
        tfp.optimizer.lbfgs_minimize(
          loss_and_flat_grad,
          initial_position=self.get_weights(),
          num_correction_pairs=self.nt_nCorrection,
          max_iterations=self.nt_maxIter,
          f_relative_tolerance=self.nt_tolFun,
          tolerance=self.nt_tolFun)
        self.nt_optimization_steps(loss_and_flat_grad)

    # def nt_optimization_steps(self, loss_and_flat_grad):
    #     lbfgs(loss_and_flat_grad,
    #           self.get_weights(),
    #           self.nt_config, Struct(), True,
    #           lambda epoch, loss, is_iter:
    #           self.logger.log_train_epoch(epoch, loss, "", is_iter))

    def np_value(tensor):
      """Get numpy value out of possibly nested tuple of tensors."""
      if isinstance(tensor, tuple):
        return type(tensor)(*(np_value(t) for t in tensor))
      else:
        return self.tensor(tensor.numpy())

    def run(optimizer):
      """Run an optimizer and measure it's evaluation time."""
      optimizer()  # Warmup.
      with timed_execution():
        result = optimizer()
      return self.np_value(result)

    def fit(self, X_u, u, v):
        self.logger.log_train_start(self)

        # Creating the tensors
        X_u = self.tensor(X_u)
        u = self.tensor(u)
        v = self.tensor(v)

        # Optimizing
        self.tf_optimization(X_u, u, v)
        self.run(self.nt_optimization(X,u,v))
        #self.nt_optimization(X_u, u, v)

        self.logger.log_train_end(self.tf_epochs + self.nt_maxIter)

    def predict(self, X_star):
        u_pred = self.model(X_star)
        return u_pred.numpy()

    def summary(self):
        return self.model.summary()

    def tensor(self, X):
        return tf.convert_to_tensor(X, dtype=self.dtype)

# Prepare data

In [ ]:
# Load Data
import scipy.io

def prep_data(path, N_train):

  data = scipy.io.loadmat(path)
        
  U_star = data['U_star'] # N x 2 x T
  P_star = data['p_star'] # N x T
  t_star = data['t'] # T x 1
  X_star = data['X_star'] # N x 2

  N = X_star.shape[0]
  T = t_star.shape[0]

  # Rearrange Data 
  XX = np.tile(X_star[:,0:1], (1,T)) # N x T
  YY = np.tile(X_star[:,1:2], (1,T)) # N x T
  TT = np.tile(t_star, (1,N)).T # N x T

  UU = U_star[:,0,:] # N x T
  VV = U_star[:,1,:] # N x T
  PP = P_star # N x T

  x = XX.flatten()[:,None] # NT x 1
  y = YY.flatten()[:,None] # NT x 1
  t = TT.flatten()[:,None] # NT x 1

  u = UU.flatten()[:,None] # NT x 1
  v = VV.flatten()[:,None] # NT x 1
  p = PP.flatten()[:,None] # NT x 1
  X = np.concatenate([x, y, t], 1)
        
  lb = X.min(0)
  ub = X.max(0)

  #TRaining DAta
  idx = np.random.choice(N*T, N_train, replace=False)
  x_train = x[idx,:]
  y_train = y[idx,:]
  t_train = t[idx,:]
  u_train = u[idx,:]
  v_train = v[idx,:]
  X_train= np.concatenate([x_train, y_train, t_train], 1)

  # Boudanry data
  x_1 = np.vstack((lb, ub))
  
  #Test DATA
  snap = np.array([100])
  x_star = X_star[:,0:1]
  y_star = X_star[:,1:2]
  t_star = TT[:,snap]

  u_star = U_star[:,0,snap]
  v_star = U_star[:,1,snap]
  p_star = P_star[:,snap]
  X_star= np.concatenate([x_train, y_train, t_train], 1)

  return X, X_train, u_train , v_train , x_1, X_star, u_star, v_star, p_star, ub, lb

# Plots

# MODEL

In [ ]:
#HYPER PARAMETERS
hp = {}
# Data size on the solution u
hp["N_u"] = 5000
# DeepNN topology (2-sized input [x t], 8 hidden layer of 20-width, 1-sized output [u]
hp["layers"] = [3, 20, 20, 20, 20, 20, 20, 2]
# Setting up the TF SGD-based optimizer (set tf_epochs=0 to cancel it)
hp["tf_epochs"] = 10
hp["tf_lr"] = 0.003
hp["tf_b1"] = 0.9
hp["tf_eps"] = None
# Setting up the quasi-newton LBGFS optimizer (set nt_epochs=0 to cancel it)
hp["nt_epochs"] = 10
hp["nt_lr"] = 0.8
hp["nt_ncorr"] = 50
hp["log_frequency"] = 10

In [ ]:
#DEFINE THE MODEL

class NavierStokesInformedNN(NeuralNetwork):
  def __init__(self, hp, logger, ub, lb):
    super().__init__(hp, logger, ub, lb)

    # Defining the two additional trainable variables for identification
    self.lambda_1 = tf.Variable([1.0], dtype=self.dtype)
    self.lambda_2 = tf.Variable([1.0], dtype=self.dtype)

  def f_model(self,X):

    l1, l2 = self.get_params()
    
    # Separating the collocation coordinates
    x_f = tf.convert_to_tensor(X[:, 0:1], dtype=self.dtype)
    y_f = tf.convert_to_tensor(X[:, 1:2], dtype=self.dtype)
    t_f = tf.convert_to_tensor(X[:, 2:3], dtype=self.dtype)

    with tf.GradientTape(persistent=True) as tape:
      #Watching the 3 inputs
      tape.watch(x_f)
      tape.watch(y_f)
      tape.watch(t_f)
      
      #Packing the inputs
      X_f = tf.stack([x_f[:,0], y_f[:,0], t_f[:,0]], axis=1)

      #Getting the prediction
      psi_and_p = self.model(X_f)
      psi = psi_and_p[:,0:1]
      p = psi_and_p[:,1:2]
      
      #Getting velocities 
      u = tape.gradient(psi, y_f)
      v = tape.gradient(tf.negative(psi), x_f)
      
      #Deriving inside
      u_x = tape.gradient(u, x_f)
      u_y = tape.gradient(u, y_f)
      v_x = tape.gradient(v, x_f)
      v_y = tape.gradient(v, y_f)
    
    u_t = tape.gradient(u, t_f)
    v_t = tape.gradient(v, t_f)

    u_xx = tape.gradient(u_x, x_f)
    u_yy = tape.gradient(u_y, y_f)
    
    v_xx = tape.gradient(v_x, x_f)
    v_yy = tape.gradient(v_y, y_f)
  
    p_x = tape.gradient(p, x_f)
    p_y = tape.gradient(p, y_f)
    
    del tape

    f_u = u_t + l1*(u*u_x + v*u_y) + p_x - l2*(u_xx + u_yy) 
    f_v = v_t + l1*(u*v_x + v*v_y) + p_y - l2*(v_xx + v_yy)
    
    return u, v, p, f_u, f_v

  def loss(self,X, u, v):
    u_pred, v_pred, p_pred, f_u_pred, f_v_pred = self.f_model(self.X)
    return tf.reduce_sum(tf.square(u - u_pred)) + \
                    tf.reduce_sum(tf.square(v - v_pred)) + \
                    tf.reduce_sum(tf.square(f_u_pred)) + \
                    tf.reduce_sum(tf.square(f_v_pred))


  def wrap_training_variables(self):
    var = self.model.trainable_variables
    var.extend([self.lambda_1, self.lambda_2])
    return var

  def get_weights(self):
    w = super().get_weights(convert_to_tensor=False)
    w.extend(self.lambda_1.numpy())
    w.extend(self.lambda_2.numpy())
    return tf.convert_to_tensor(w, dtype=self.dtype)

  def set_weights(self, w):
    super().set_weights(w)
    self.lambda_1.assign([w[-2]])
    self.lambda_2.assign([w[-1]])

  def get_params(self, numpy=False):
    l1 = self.lambda_1
    l2 = self.lambda_2
    if numpy:
        return l1.numpy()[0], l2.numpy()[0]
    return l1, l2

  def fit(self, X, u, v):
    self.X =  tf.convert_to_tensor(X, dtype=self.dtype)
    super().fit(X, u, v)

  def predict(self, X): 
    u_star, v_star, p_star, _ , _ = self.f_model(X)
    return u_star, v_star, p_star


# LOGGER

In [ ]:
import json
import tensorflow as tf
import time
from datetime import datetime
import tensorflow_probability as tfp


class Logger(object):
    def __init__(self, hp):
        print("Hyperparameters:")
        print(json.dumps(hp, indent=2))
        print()

        print("TensorFlow version: {}".format(tf.__version__))
        print("Eager execution: {}".format(tf.executing_eagerly()))
        print("GPU-accerelated: {}".format(tf.test.is_gpu_available()))

        self.start_time = time.time()
        self.prev_time = self.start_time
        self.frequency = hp["log_frequency"]

    def get_epoch_duration(self):
        now = time.time()
        edur = datetime.fromtimestamp(now - self.prev_time) \
            .strftime("%S.%f")[:-5]
        self.prev_time = now
        return edur

    def get_elapsed(self):
        return datetime.fromtimestamp(time.time() - self.start_time) \
                .strftime("%M:%S")

    def get_error_u(self):
        return self.error_fn()

    def set_error_fn(self, error_fn):
        self.error_fn = error_fn

    def log_train_start(self, model, model_description=False):
        print("\nTraining started")
        print("================")
        self.model = model
        if model_description:
            print(model.summary())

    def log_train_epoch(self, epoch, loss, custom="", is_iter=False):
        if epoch % self.frequency == 0:
            name = 'nt_epoch' if is_iter else 'tf_epoch'
            print(f"{name} = {epoch:6d}  " +
                  f"elapsed = {self.get_elapsed()} " +
                  f"(+{self.get_epoch_duration()})  " +
                  f"loss = {loss:.4e}  " + custom)

    def log_train_opt(self, name):
        print(f"-- Starting {name} optimization --")

    def log_train_end(self, epoch, custom=""):
        print("==================")
        print(f"Training finished (epoch {epoch}): " +
              f"duration = {self.get_elapsed()}  " +
              f"error = {self.get_error_u():.4e}  " + custom)

# TRAIN THE MODEL

In [ ]:
# Getting the data
path='/cylinder_nektar_wake.mat'
X, X_train, u_train , v_train , x_1, X_star, u_star, v_star, p_star, ub, lb = prep_data(path, hp["N_u"])
lambdas_star = (1.0, 100) #true values

# Creating the model
logger = Logger(hp)
pinn = NavierStokesInformedNN(hp, logger, ub, lb)


# Defining the error function and training
def error():
  l1, l2 = pinn.get_params(numpy=True)
  l1_star, l2_star = lambdas_star
  error_lambda_1 = np.abs(l1 - l1_star) / l1_star
  error_lambda_2 = np.abs(l2 - l2_star) / l2_star
  return (error_lambda_1 + error_lambda_2) / 2
logger.set_error_fn(error)
pinn.fit(X_train, u_train, v_train)



Hyperparameters:
{
  "N_u": 5000,
  "layers": [
    3,
    20,
    20,
    20,
    20,
    20,
    20,
    2
  ],
  "tf_epochs": 10,
  "tf_lr": 0.003,
  "tf_b1": 0.9,
  "tf_eps": null,
  "nt_epochs": 10,
  "nt_lr": 0.8,
  "nt_ncorr": 50,
  "log_frequency": 10
}

TensorFlow version: 2.4.1
Eager execution: True
GPU-accerelated: True

Training started
-- Starting Adam optimization --
tf_epoch =      0  elapsed = 00:06 (+06.9)  loss = 5.7296e+03  
-- Starting LBFGS optimization --


ValueError: ignored

In [ ]:
# Getting the model predictions, from the same (x,t) that the predictions were previously gotten from
u_pred, v_pred, p_pred = pinn.predict(X_star)
lambda_1_pred, lambda_2_pred = pinn.get_params(numpy=True)

In [ ]:
print("l1: ", lambda_1_pred)
print("l2: ", lambda_2_pred)